# What is metadynminer

Welcome to Metadynminer, our Python package developed to make analysis of metadynamics simulations easy and user-friendly. This Jupyter notebook is designed to introduce metadynminer to new users and also to explain the algorithms used by metadynminer to some extent. 

Just a reminder, if you are using this notebook in our online service to process your own results, do not forget to download all files you may have created to your PC for later use. 

In [ ]:
import metadynminer as mm
print(f"Loaded metadynminer version {mm.__version__}.")

You can uncomment and run the next line to enable ```%matplotlib widget``` for better interactivity with the graphs, if you want to. It is not necessary in most cases. To turn it back off, you may need to restart the kernel of this notebook. 

In [ ]:
# %matplotlib widget

# Load your HILLS file 

First upload your HILLS file and load it with the line below. If the name of your HILLS file is not the default "HILLS", specify it to the ```name``` keyword. 

If you do not have your own HILLS file, you can try our code with our HILLS file from the oxytocin metadynamics simulation used for testing and demonstration, using 
```python
hills = mm.Hills(name="oxytocin")
```

Metadynminer automatically detects which CVs are periodic, however, if it fails to detect periodicity correctly, 
you can specify the periodicity of your CVs as boolean Python list
(for example, if you have two periodic CVs, set ```periodic = [True, True]```): 
```python
hills = mm.Hills(name="HILLS", periodic=[True, True])
```

Default periodicity of periodic CVs is set from -$\pi$ to $\pi$. If your CVs have different periodicity, you have to specify it to the respective keyword ```cv1per```, ```cv2per``` or ```cv3per```. 

An example for CV 1 with periodicity from 0 to 2$\pi$: 
```python
import numpy as np
hills = mm.Hills(name="HILLS", periodic=[True, True], cv1per=[0, 2*np.pi])
```

In [ ]:
hills = mm.Hills(name="HILLS")

# Plot CVs and heights of the hills

You can show the evolution of your molecular system by plotting values of the collective variables in time. For this, use the ```plot_CV``` method. Specify which CV should be plotted to the ```CV``` keyword. If you want to save and download the image with the plot later, specify the name for the file to the  ```png_name``` keyword, for example: 
```python
hills.plot_CV(png_name="CV_1_plot.png", CV=1)
```
Axis labels will be generated automatically based on the information inside the HILLS file. If you want to use your own labels, set them as strings to ```xlabel```, ```ylabel``` keywords:
```python
hills.plot_CV(png_name="CV_1_plot.png", CV=1, ylabel="CV 1")
```
You can also change the time unit to be shown as necessary: ```tu="ns"``` etc. 

To export the image in publication quality, you might want to use other keywords to set the size and dpi of the image, for example: ```dpi=300```, ```image_size=[16,9]``` and ```image_size_unit="cm"```

In [ ]:
hills.plot_CV(png_name=None, CV=1, tu="ns")

In [ ]:
hills.plot_CV(png_name=None, CV=2, tu="ns")

In well-tempered metadynamics, it might be also usefull to visualise the heights of the hills added during the simulation, 
as these tend to decrease over time when the system revisits already explored parts of the CV space: 

In [ ]:
hills.plot_heights(png_name=None, energy_unit="kJ/mol", title="heights of hills")

# Calculate the free energy surface from hills file

You have two options to choose from here: 
If you use ```original=False```, the FES will be calculted by the Hillsum algorithm - only one Gaussian hill will be precalculated 
and then moved and scaled and added to the FES. This is less precise, but much faster than calculating each Gaussian explicitly. 
This is ideal for casual visualisation purposes. 

If you use ```original=True```, each Gaussian hill will be calculated explicitly, which is more computationally demanding, but also exact. 
This method is giving the same results as the ```sum_hills``` function in Plumed (tested with Plumed v2.8.0)

If you don't need high resolution FES and/or you find the calculation too slow, you can decrease the ```resolution``` of the FES. 

In [ ]:
fes = mm.Fes(hills, original=False, resolution=256)

Then you can visualise the FES. Keywords controlling the image size, dpi, name etc. are the same as for the CV plots. 

For 2D and 3D FESs, you can change the colormap to any of the matplotlib colormaps, for example ```cmap="rainbow"```. 

On 2D and 3D FESs, you can also control the contours or isosurfaces to be shown, either as a list specifying the values of free energy to be visualised as here: ```levels=[20,40,60]```, or by specifying the spacing between each contour/isosurface: ```contours_spacing=20```. 

Sometimes it is useful to change the minimum or maximum value of free energy to be colored with keywords ```vmin```, ```vmax```.

To plot only a slice of the FES, you can use keywords ```xlim=[x_minimum, x_maximum]```, ```ylim=[y_minimum, y_maximum]``` with a list of the beginning and the end of the slice.

For 3D FESs, it may be useful to change the opacity of the isosurfaces to get optimal visibility: ```opacity=0.2```. 


For example:
```python
fes.plot(png_name=None, contours=True, cmap = "RdYlBu_r", 
             energy_unit="kJ/mol", xlabel=None, ylabel=None, zlabel=None, label_size=12, clabel_size = 12,
             image_size=[9,6], image_size_unit="in", dpi=100, vmin = 0, vmax = None, levels=[20,40,60,80,100])
```

In [ ]:
fes.plot( cmap = "RdYlBu_r")

# Identify free energy minima

You have two options: 
If you set the keyword ```precise=True```, the local minima will be identified by an algorithm which: 
1. finds all local minima, even very shallow and probably unimportant minima, 
2. each point on the FES will be assigned to the minimum the system would most likely go to, 
   if it only follows the gradient of free energy, and 
3. free energy value of minima will be calculated from each point on FES assigned to the respective minima. 
   This results in more precise free energy values, as it accounts for the width of the minimum as well. 
   For this calculation the unit of free energy (```energy_unit="kJ/mol"``` or ```energy_unit="kcal/mol"```) and 
   the thermodynamical temperature (```temp```) of the simulation must be supplied. 
   This algorithm does not use the ```nbins``` keyword. 

Example:
```python
minima = mm.Minima(fes, precise=True, temp=300.0, energy_unit="kJ/mol")
```

If you set ```precise = False```, the method will use the original algorithm from the metadynminer package for R. 
In this algorithm the FES is first divided to number of bins (can be set with option ```nbins```, default is 8 for 1D FES, $8\times 8$ for 2D FES or $8\times 8 \times 8$ for 3D FES)
and the absolute minima is found for each bin. Then the algorithm checks 
if this point is really a local minimum by comparing to the surrounding points of FES.
This algorithm only accounts for the depth of each minima, which is less precise, but usually sufficient. 

In some cases, this algorithm is the prefered one, 
because on some free energy landscapes the total number of local free energy minima can reach tens of thousands, 
which makes the calculation using precise algorithm slow and impractical. 

Example: 
```python
minima = mm.Minima(fes, precise=False, nbins=8)
```

In [ ]:
minima = mm.Minima(fes, precise=True, temp=300.0, energy_unit="kJ/mol")

Print the list of the local minima:

In [ ]:
minima.minima

You can visualise the FES with the local minima as letters. All keywords are analogous as for the ```fes.plot()``` function.
The color of the letters changes automatically to ensure their good visibility, but if you want to override this behaviour, provide a matplotlib color like this: ```color="black"```. 

In [ ]:
# plot the free energy surface with minima
minima.plot(contours_spacing=20)

# Construct free energy profile

In metadynamics simulations, it is useful to visualise how the FES was changing during the simulation, which can suggest (not prove) whether the FES is converged or not and the simulation should be prolonged. 
Here, you should provide the list of minima as well as the hills objects:

In [ ]:
prof = mm.FEProfile(minima,hills)

Plot the free energy profile:

In [ ]:
prof.plot(legend=True, png_name=None)

# Other optional methods for analysis

You can remove one CV from an existing FES (for example to make visualisation of 3D FES easier). For this purpose use the ```Fes.remove_CV()``` method, which will return a new FES with one CV removed. The algorighm converts the FES to probabilities, then sums the probabilities along the given ```CV``` to be removed and then converts these sums back to free energy values. Because of this, you should provide the temperature of the simulation as well as the unit of free energy used in HILLS file. ```temp=300.0``` Kelvin and ```energy_unit="kJ/mol"``` are the default values. 

In [ ]:
fes_CV1 = fes.remove_CV(CV=2, temp=300.0, energy_unit="kJ/mol")

You can work with the new fes object in the same way as with the original one, e. g. find minima, plot them etc.:

In [ ]:
minima_1 = mm.Minima(fes_CV1)
minima_1.minima

In [ ]:
minima_1.plot()

Another alternative way to visualise 2D FES is by creating a surface plot. This method only works for 2D FESs. This works best together with ```%matplotlib widget``` turned on (you can find the line at the beginning of this notebook). If you find the animation too slow, it may be necessary to decrease the resolution of the FES. 

In [ ]:
fes.surface_plot()

Visualising 3D FES can be challenging. One possible way is by plotting different isosurfaces with specific free energy values in 3D space, as in the ```Fes.plot()``` and ```Minima.plot()``` methods. Another way is by creating an animation showing different isosurfaces at diferent times with ```Fes.make_gif()``` method and it's analog ```Minima.make_gif()```. These methods are only available for 3D FESs. 

In [ ]:
fes.make_gif(gif_name="fes_animation.gif", energy_unit="kJ/mol",
                  opacity=0.2, frames=64)

The resulting animation (if available). Maybe you will need to refresh this page of your browser for the animation to be shown:

<img src="fes_animation.gif" width="750" align="center"/>

It is also possible to create an animation of the flooding of the free energy surface as the simulation ran. This can be useful to visualise the convergence of the FES and the evolution of the CV values during the simulation at the same time. 

In [ ]:
fes.flooding_animation(step=1000, gif_name="flooding.gif", fps=10, enable_loop=True, 
                       contours_spacing=10, with_minima=True, use_vmax_from_end=False)

Again, maybe you will need to refresh this page of your browser for the resulting animation to be shown:

<img src="flooding.gif" width="750" align="center"/>

# Reweighting

There is also more precise (and sometimes more complicated) way to obtain the free energy surface, commonly reffered to as *reweighting*. Here, we show Tiwary's reweighting. In principle, FES can be calculated with respect to any CV by calculating the free energy of states with some value from their relative population compared to other states. That is quite simple for unbiased simulations. In metadynamics, some states are discouraged more than others by the bias potential, which must be accounted for during reweighting. In other words, for states that were populated even though they were discouraged by bias potential, the relative population has higher weight compared to not-so-discouraged states. Reweighting can be used to calculate the FES for the same CVs that were biased by metadynamics, but also with respect to different CVs, as long as you can provide the values of the CVs during simulation. You can calculate these values for example with ```plumed driver```. 

In [ ]:
import numpy as np

Specify temperature of simulation, energy unit in HILLS file and resolution of resulting reweighted FES:

In [ ]:
temperature = 300.0
energy_unit="kJ/mol"
resolution=50 
maxfes = 75

In [ ]:
if energy_unit == "kJ/mol":
    kT = 8.314*temperature/1000
elif energy_unit == "kcal/mol":
    kT = 8.314*temperature/1000/4.184
biasfactor = float(hills.biasf[0])
nsteps=50
outfes = mm.Fes(hills, resolution=resolution, calculate_new_fes=False)

Here, you should provide one other output file from metadynamics simulation (common name being ```COLVAR```) containing the total bias potential present applied to the system at each sampled  moment. If you also calculated the reweighting factor on the fly (with ```CALC_RCT``` keyword), the following calculation will be a bit more simple. The default name of the column containing the reweighting values is ```metad.rbias```. 

If your ```COLVAR``` file does not contain the ```rbias``` values, but only ```metad.bias```, you should calculate the Tiwary's correction manually - skip to the subsection 8.2. 

In [ ]:
colvar = np.loadtxt("COLVAR")

## Algorithm using the reweighting factor ```metad.rbias``` calculated on-the-fly

In [ ]:
# excluding values where time == 0.0, because otherwise when working with concatenated simulations, the same state would be sampled twice 
# (once at the end of previous simulation part and then at the begining of the continuation)
colvar1 = colvar[colvar[:,0]!=0.0,1] # colvar[:,1] contains values of CV1 during simulation
colvar2 = colvar[colvar[:,0]!=0.0,2] # colvar[:,2] contains values of CV2 during simulation
rbias = colvar[colvar[:,0]!=0.0,4] # colvar[:,3] contains the metad.rbias column

nsamples = colvar[colvar[:,0]!=0.0].shape[0]
step = np.arange(0, nsamples)*nsteps/(nsamples+1)
ix = (resolution*(colvar1-outfes.cv1min)/(outfes.cv1max-outfes.cv1min)).astype(int)
iy = (resolution*(colvar2-outfes.cv2min)/(outfes.cv2max-outfes.cv2min)).astype(int)
probabilities = np.zeros((outfes.fes.shape))
ebias = np.exp(rbias/kT)
for i in range(int(nsamples)):
    probabilities[ix[i], iy[i]] = probabilities[ix[i],iy[i]] + ebias[i]

outfes.fes = -kT*np.log(probabilities)
outfes.fes = outfes.fes - np.min(outfes.fes)
outfes.fes[outfes.fes>maxfes] = maxfes
outfes.plot()

## Algorithm using ```metad.bias``` calculating Tiwary's correction post *ex post*

In [ ]:
hills.hills.shape

In [ ]:
step = np.arange(1,nsteps+1)*hills.cv1.shape[0]/nsteps
s1 = np.zeros((step.shape))
for i in range(len(s1)):
    # print progress 
    print(f"Progress: {((i+1)/len(s1)):.2%} finished. ", end="\r")
    if i == 0:
        s1[i] = np.sum(np.exp((-mm.Fes(hills, original=False, print_output=False, 
                                   time_max=step[i], subtract_min=False).fes/kT)*((biasfactor-1)/biasfactor)))
    else:
        s1[i] = s1[i-1] + np.sum(np.exp((-mm.Fes(hills, original=False, print_output=False, 
                                   time_min=step[i-1], time_max=step[i], subtract_min=False).fes/kT)*((biasfactor-1)/biasfactor)))

In [ ]:
s2 = np.zeros((step.shape))
for i in range(len(s2)):
    # print progress 
    print(f"Progress: {((i+1)/len(s1)):.2%} finished. ", end="\r")
    if i == 0:
        s2[i] = np.sum(np.exp((-mm.Fes(hills, original=False, print_output=False, 
                                   time_max=step[i], subtract_min=False).fes/kT/biasfactor)*((biasfactor-1)/biasfactor)))
    else:
        s2[i] = s2[i-1] + np.sum(np.exp((-mm.Fes(hills, original=False, print_output=False, 
                                   time_min=step[i-1], time_max=step[i], subtract_min=False).fes/kT/biasfactor)*((biasfactor-1)/biasfactor)))

In [ ]:
ebetac = s1/s2

# excluding values where time == 0.0, because otherwise when working with concatenated simulations, the same state would be sampled twice 
# (once at the end of previous simulation part and then at the begining of the continuation)
colvar1 = colvar[colvar[:,0]!=0.0,1] # colvar[:,1] contains values of CV1 during simulation
colvar2 = colvar[colvar[:,0]!=0.0,2] # colvar[:,2] contains values of CV2 during simulation
bias = colvar[colvar[:,0]!=0.0,3] # colvar[:,3] contains the metad.bias column

nsamples = colvar[colvar[:,0]!=0.0].shape[0]
step = (np.arange(0, nsamples)*nsteps/(nsamples+1)).astype(int)
ix = (resolution*(colvar1-outfes.cv1min)/(outfes.cv1max-outfes.cv1min)).astype(int)
iy = (resolution*(colvar2-outfes.cv2min)/(outfes.cv2max-outfes.cv2min)).astype(int)
ebetac = np.repeat(ebetac, int(bias.shape[0]/ebetac.shape[0]))
ebias = np.exp(bias/kT)/ebetac
probabilities = np.zeros((outfes.fes.shape))
for i in range(int(nsamples)):
    probabilities[ix[i], iy[i]] = probabilities[ix[i],iy[i]] + ebias[i]

In [ ]:
outfes.fes = -kT*np.log(probabilities)
outfes.fes = outfes.fes - np.min(outfes.fes)
outfes.fes[outfes.fes>maxfes] = maxfes
outfes.plot()